# GeekBrains, Recommendation Systems
# Lesson 2 Homework

**Импорт библилотек**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

**Датасет**

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [23]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,[ 821867 834484 856942 865456 889248 ...,"[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"


**Делим на трейн и тест**

In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

### Решение Задания 1

In [30]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    # Используем параметр p
    recs = np.random.choice(items,
                            size=n,
                            replace=False,
                            p=weights)
    
    return recs.tolist()

In [27]:
%%time

# your_code
# Считаем продажи каждого товара, берем логарифм
# Полученные значения делаем положительными и приводим к единичной сумме
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['sales_value'] = np.log2(item_weights['sales_value'] + 0.001)
item_weights['sales_value'] = item_weights['sales_value'] - item_weights['sales_value'].min()
item_weights['sales_value'] = item_weights['sales_value'] / item_weights['sales_value'].sum()
item_weights = item_weights.rename(columns={'sales_value': 'weight'})
item_weights

Wall time: 272 ms


,item_id,weight
0,25671,0.000012
1,26081,0.000008
2,26093,0.000009
3,26190,0.000009
4,26355,0.000009
...,...,...
86860,17381856,0.000000
86861,17382205,0.000011
86862,17383227,0.000010
86863,17827644,0.000010


In [29]:
%%time

result['w_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))
result.head(2)

Wall time: 11.4 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[9419419, 573302, 853690, 1133754, 978657]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1030522, 903398, 854456, 1886755, 975940]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
# your_code

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code